# Análisis Exploratorio de Datos (EDA) - Online Retail

En este cuaderno, realizaremos un **análisis paso a paso** de un dataset de facturación que contiene columnas como:
 - **InvoiceNo**: Número de factura
 - **StockCode**: Código del producto
 - **Description**: Descripción del producto
 - **Quantity**: Cantidad vendida (puede ser negativa en devoluciones)
 - **InvoiceDate**: Fecha de la factura
 - **UnitPrice**: Precio unitario de cada artículo
 - **CustomerID**: ID del cliente (puede haber nulos si no se registró)
 - **Country**: País donde se realizó la compra

El objetivo es **entender** el comportamiento de las ventas, los países más relevantes, los productos más comprados, y otras métricas de interés.

In [1]:
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
!unzip ecomerce_customers_segmentation.zip

--2025-09-15 17:07:02--  https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip [following]
--2025-09-15 17:07:02--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7548686 (7.2M) [application/zip]
Saving to: ‘ecomerce_customers_segmentation.zip’

ecomerce_customers_ 100%[===================>]

## 1. Importar librerías y cargar datos
Asegúrate de tener el archivo CSV en la misma carpeta, o ajusta la ruta de `pd.read_csv` según tu necesidad.

In [2]:
import pandas as pd
import numpy as np

# Carga del dataset (ajusta el nombre a tu archivo)
df = pd.read_csv('data.csv',encoding="latin-1") # Puedes necesitar encoding='latin1' si hay caracteres especiales
pd.set_option('display.max_columns', None)
# Mostramos las primeras filas para inspección
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. Información general del dataset
Aquí veremos:
- Dimensiones del DataFrame
- Tipos de datos
- Presencia de valores nulos
- Estadísticos descriptivos de las columnas numéricas

In [3]:
df.shape





(541909, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# Estadísticos descriptivos
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


**Observación**: Algunas facturas pueden ser devoluciones si `Quantity` es negativa. Además, puede haber filas sin `CustomerID`.

In [6]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


## 3. Limpieza básica y transformación
### 3.1. Revisar y eliminar posibles filas con datos problemáticos
 - Filas donde `Description` esté vacía
 - Filas con `Quantity == 0` (puede haber poco sentido en un registro así)
 - Filas con `UnitPrice <= 0` (podría ser error o promoción anómala)
 - Convertir la columna `InvoiceDate` a formato fecha, para luego analizar tiempos

In [7]:
df = df.dropna(subset=["Description"])
df = df[df["Quantity"] != 0]
df = df[df["UnitPrice"] != 0]
df["InvoiceDate"] =pd.to_datetime(df["InvoiceDate"])

In [8]:
df["InvoiceDate"][4]

Timestamp('2010-12-01 08:26:00')

## 4. Creación de columna "TotalPrice"
Para conocer el valor total de cada línea de venta, calculamos `Quantity * UnitPrice`.

In [9]:
df["TotalPrice"] = df["Quantity"]*df["UnitPrice"]

In [10]:
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## 5. Análisis paso a paso
A continuación, veremos distintos análisis que **paso a paso** nos ayudarán a entender mejor el negocio.

### 5.1. ¿Cuántas transacciones totales hay y cuántos clientes diferentes?
Esto da un primer vistazo al tamaño del negocio en términos de facturas y base de clientes.

In [ ]:
len(df)


539394

In [ ]:
df["CustomerID"].value_counts()

,count
CustomerID,
17841.0,7983
14911.0,5901
14096.0,5128
12748.0,4641
14606.0,2782
...,...
16138.0,1
16765.0,1
16144.0,1


### 5.2. ¿Cuáles son los países más frecuentes en las ventas?
Podemos contar cuántas líneas de factura vienen de cada país. Esto ayuda a entender nuestro alcance geográfico.

In [11]:
df["Country"].value_counts()

,count
Country,
United Kingdom,492981
Germany,9493
France,8556
EIRE,8192
Spain,2532
Netherlands,2367
Belgium,2069
Switzerland,2001
Portugal,1519


> Tip: Si queremos ver el total en **monetario** por país, podemos agrupar por `Country` y sumar `TotalPrice`.

In [ ]:
df.groupby("Country")["TotalPrice"].sum()

,TotalPrice
Country,
Australia,137077.270
Austria,10154.320
Bahrain,548.400
Belgium,40910.960
Brazil,1143.600
Canada,3666.380
Channel Islands,20086.290
Cyprus,12946.290
Czech Republic,707.720


### 5.3. ¿Cuáles son los productos más vendidos?
Aquí vamos a usar `Description` para ver cuántas líneas se han vendido (o la suma total de `Quantity`).

In [14]:
# Top 10 productos por cantidad total vendida
df.groupby("Description")["TotalPrice"].sum().sort_values(ascending=False)


,TotalPrice
Description,
DOTCOM POSTAGE,206245.480
REGENCY CAKESTAND 3 TIER,164762.190
WHITE HANGING HEART T-LIGHT HOLDER,99668.470
PARTY BUNTING,98302.980
JUMBO BAG RED RETROSPOT,92356.030
...,...
Bank Charges,-7175.639
CRUK Commission,-7933.430
Adjust bad debt,-11062.060


### 5.4. ¿Cuál es el total de ingresos (revenue) y el ticket promedio?
 - El ingreso total es la suma de **TotalPrice**.
 - El ticket promedio podemos calcularlo como el promedio de **TotalPrice** dentro de cada factura (o línea).

In [ ]:
# ingreso total
df["TotalPrice"].sum()
# total promedio
df["TotalPrice"].mean()

np.float64(18.071665487565674)

> Para un ticket promedio a nivel de **factura**, debemos agrupar por `InvoiceNo`, sumar `TotalPrice` y luego promediar.

In [ ]:
# Ticket promedio por factura
df.groupby("InvoiceNo")["TotalPrice"].mean()


,TotalPrice
InvoiceNo,
536365,19.874286
536366,11.100000
536367,23.227500
536368,17.512500
536369,17.850000
...,...
C581484,-168469.600000
C581490,-16.265000
C581499,-224.690000


### 5.6. ¿Tenemos devoluciones?
Como `Quantity` puede ser negativa, **representa** devoluciones. Veamos cuántas hay y el impacto en valor.

In [20]:
# Filtrar devoluciones (Quantity < 0)
df.loc[df["TotalPrice"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,-27.50
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,-6.96
...,...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,-9.13
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,-224.69
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,-54.75
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,-1.25


In [23]:
negatives_sum = df.loc[df["TotalPrice"] < 0, "TotalPrice"].sum()
print(negatives_sum)

-918936.6099999999


### 5.7. ¿Cuántos clientes realizan compras repetidas?
Podemos ver cuántos clientes compran más de una vez.

In [24]:
df["CustomerID"].value_counts()

,count
CustomerID,
17841.0,7983
14911.0,5901
14096.0,5128
12748.0,4641
14606.0,2782
...,...
16138.0,1
16765.0,1
16144.0,1


## 6. Posibles siguientes pasos
- **Crear segmentaciones** (p. ej. RFM: Recency, Frequency, Monetary) para clasificar clientes.
- **Analizar descuentos/promos** si tuvieras columnas extra.
- **Fusionar** con datos de marketing para calcular CAC (Customer Acquisition Cost) y LTV (Lifetime Value), etc.
- **Explorar** variables estacionales o días de la semana con mayor venta.

Con este **análisis paso a paso**, se obtiene un primer panorama de:
 - Cantidad de facturas y clientes.
 - Productos más vendidos.
 - Ingresos por país.
 - Montos totales y tickets promedio.
 - Distribución mensual de ventas.
 - Devoluciones y recurrencia de clientes.

¡A medida que se adquieran más habilidades en Python y análisis de datos, se pueden añadir gráficos, segmentaciones avanzadas y modelos predictivos!